In [17]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import optimize

from sklearn.metrics import accuracy_score, f1_score
from pymoo.core.problem import Problem
from evoman.environment import Environment
from fitness_functions import original_fitness, individual_gain
from utils import simulation, verify_solution, init_env, run_pymoo_algorithm, initialise_script
import math


In [18]:
solutions = []
for file in os.listdir('solutions_beats_5_enemies'):
    if file.startswith('pymoo'):
        new_solution = []
        with open(f'solutions_beats_5_enemies/{file}') as f:
            solutions.append(f.read().splitlines())
            
solutions = np.array(solutions, dtype=float)

In [31]:
seed = 42
POP_SIZE = solutions.shape[0]
ENEMIES = [1, 2, 3, 4, 5, 6, 7, 8]
n_hidden_neurons = 10


In [35]:
class objectives(Problem):
    enemies: list[int]
    env: Environment

    def __init__(self, env: Environment, n_genes: int, enemies: list[int], n_objectives):
        self.env = env
        self.enemies = enemies
        super().__init__(n_var=n_genes, n_obj=n_objectives, xl=-1, xu=1, type_var=float)

    def _evaluate(self, x: list[np.array], out, *args, **kwargs):
        if POP_SIZE != len(x):
            print(f"WARNING: POP_SIZE != len(x) in evaluation step (this happens sometimes do not see why)\n"
                  f"pop size: {POP_SIZE}; len x:{len(x)}")
        # Initialize
        dict_enemies = {}
        # Get fitness for each enemy
        for enemy in self.enemies:
            self.env.update_parameter('enemies', [enemy])

            dict_enemies[enemy] = []
            for individual_id in range(len(x)):
                dict_enemies[enemy].append(
                    simulation(self.env, x[individual_id], inverted_fitness=True, fitness_function=individual_gain))

        # Return fitness outputs for enemies
        objectives_fitness = {
            "objective_hard": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [1, 6]]) for ind_id in
                               range(len(x))],
            "objective_medium": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [2, 5, 8]]) for ind_id in
                                 range(len(x))],
            "objective_easy": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [3, 4, 7]]) for ind_id in
                               range(len(x))],
        }

        out["F"] = np.column_stack([objectives_fitness[key] for key in objectives_fitness.keys()])
        return out

In [66]:
env, n_genes = init_env('local_search_test', ENEMIES, n_hidden_neurons)
env.update_parameter('multiplemode', 'no')
problem = objectives(
        env=env,
        n_genes=n_genes,
        enemies=ENEMIES,
        n_objectives=3)

In [75]:
x_curr = solutions[[0],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(x_curr, {})['F']
f_best = np.copy(f_curr)

pop size: 23; len x:1


In [77]:
# This is the function that needs to be defined, this is how you find neighbors of solution x
# For now, it gives a neighborhood where every gene has one 'mutation' or difference (the np.random.normal)
# I could not find clear examples of algorithms for the neighbor generating

def neighborhood(x):
    nbrhood = []
    for i in range(x.shape[1]):
        new_nbr = np.copy(x)
        new_nbr[i] = new_nbr[i] * np.random.normal()
        nbrhood.append(new_nbr)
    
    return nbrhood

In [78]:
done = False
solutionsChecked = 0

while not done:
    Neighborhood = neighborhood(x_curr)
    for s in Neighborhood:
        solutionsChecked += 1
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except:
            print("Infeasible solution evaluated")
            continue
        
        if eval_s[0].mean() > f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
    
    if np.array_equal(f_best, f_curr):
        done = True
    else:
        x_curr = np.copy(x_best)
        f_curr = np.copy(f_best)
        
        print(f"Total number of solutions checked: {solutionsChecked}")
        print(f"Best value found so far: {f_best}")
        
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"Best value found: {f_best}")
print(f"Best solution: {x_best}")

pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
pop size: 23; len x:1
Final number of solutions checked: 10
Best value found: [[0.00691027 0.00694143 0.00692763]]
Best solution: [[ 1.90818695e-05  2.00726590e-01 -1.44952210e-02  4.74959274e-01
   1.62479146e-01  3.73675244e-02  1.68265087e-01  2.71761470e-01
   2.76711926e-01 -3.87037040e-02 -5.11533295e-02 -1.75783642e-01
  -3.12610512e-01 -7.40310233e-03  2.61309281e-02 -1.47625062e-01
   5.13403976e-02  1.38474102e-03 -3.69150021e-02  3.68806990e-02
   1.03603349e-01 -8.20620068e-01 -4.95528581e-02  3.06391235e-01
  -2.08673031e-03 -2.85718941e-01  8.62343484e-03  4.28708446e-02
  -2.24159054e-02 -1.45038387e-02  1.20654964e-02 -8.28165913e-03
   8.19431710e-04 -2.70086255e-02  7.93383715e-02  3.22462779e-01
  -2.42860976e-01  1.09622721e-01 -2.46732863e-03  2.93734874e-01
   1.6553091